In [4]:
'''textrank python 实现二'''
import jieba.posseg as pseg
import numpy as np
class TextRank(object):
    def __init__(self,sentence,window,d,maxiter):
        self.sentence=sentence
        self.alpha=d
        self.window=window
        self.edge_dict={}
        self.maxiter=maxiter
    #分词
    def cutSentence(self):
#         jieba.load_userdict("user_dict.txt")
        tag_filter=["a","n","v","d"]
        seg_word=pseg.cut(self.sentence)
        self.word_list=[s.word for s in seg_word if s.flag in tag_filter]
        print(self.word_list)
    #根据窗口，构建每个节点的相邻节点
    def createNodes(self):
        tmp_list=[]
        words_len=len(self.word_list)
        for index,word in enumerate(self.word_list):
            if word not in self.edge_dict:
                tmp_list.append(word)
                tmp_set=set()
                left=index-self.window+1
                right=index+self.window
                if left<0:left=0
                if right>=words_len:right=words_len
                for i in range(left,right):
                    if i==index:continue
                    tmp_set.add(self.word_list[i])
                self.edge_dict[word]=tmp_set
    #构造矩阵
    def createMatrix(self):
        self.matrix=np.zeros([len(set(self.word_list)),len(set(self.word_list))])
        self.word2id={}
        self.id2word={}
        for i,v in enumerate(set(self.word_list)):
            self.word2id[v]=i
            self.id2word[i]=v
        for key in self.edge_dict.keys():
            for w in self.edge_dict[key]:
                #"是"：{["是","全国",1],["是","失业率",1],["",""].......}
                self.matrix[self.word2id[key],self.word2id[w]]=1
                self.matrix[self.word2id[w],self.word2id[key]]=1
        #归一化(分母位置有个除以权重和)
        for j in range(self.matrix.shape[1]):
            sum=0
            for i in range(self.matrix.shape[0]):
                sum+=self.matrix[i][j]
            for i in range(self.matrix.shape[0]):
                self.matrix[i][j]/=sum
    #计算权重
    def calPR(self):
        self.PR=np.ones([len(set(self.word_list)),1])#每个边权重初始化为1，
        for i in range(self.maxiter):
            #np.dot(self.matrix,self.PR)：相当于是每个单词的邻居单词的边权重之和
            self.PR=(1-self.alpha)+self.alpha*np.dot(self.matrix,self.PR)
    def printResult(self):
        word_PR={}
        for i in range(len(self.PR)):
            word_PR[self.id2word[i]]=self.PR[i][0]
        res=sorted(word_PR.items(),key=lambda x:x[1],reverse=True)
        print(res)
if __name__ == '__main__':
    s = '程序员(英文Programmer)是从事程序开发、维护的专业人员。一般将程序员分为程序设计人员和程序编码人员，但两者的界限并不非常清楚，特别是在中国。软件从业人员分为初级程序员、高级程序员、系统分析员和项目经理四大类。'
    tr = TextRank(s, 3, 0.85, 700)
    tr.cutSentence()
    tr.createNodes()
    tr.createMatrix()
    tr.calPR()
    tr.printResult()
        
    

['程序员', '是', '从事', '维护', '专业', '人员', '一般', '将', '程序员', '分为', '程序设计', '人员', '程序', '编码', '人员', '两者', '界限', '不', '非常', '清楚', '特别', '是', '软件', '分为', '程序员', '程序员', '系统分析员', '项目经理']
[('人员', 2.0989934884816974), ('程序员', 1.9729839490141816), ('是', 1.3055211997946863), ('分为', 1.0883357808318896), ('不', 0.9440603915103938), ('非常', 0.9425333709846045), ('两者', 0.9339700300086595), ('清楚', 0.9324671301570883), ('界限', 0.9277842524279716), ('特别', 0.9252570305379795), ('程序设计', 0.9062573419181348), ('从事', 0.9033774338714338), ('软件', 0.9029204113307728), ('专业', 0.9012332879639415), ('维护', 0.8968430615485421), ('一般', 0.8953975819588343), ('将', 0.8900408883910683), ('编码', 0.7333715175911032), ('程序', 0.7287827283293604), ('项目经理', 0.584934561673827), ('系统分析员', 0.584934561673827)]


In [5]:
#!/usr/bin/env python3
# coding: utf-8
# File: doc_rank.py
# Author: lhy<lhy_in_blcu@126.com,https://huangyong.github.io>
# Date: 18-7-9
#这个计算公式实现更好理解一点
import jieba.posseg as pseg
import os
from collections import Counter
from collections import defaultdict
import sys
import math


class textrank_graph:
    def __init__(self):
        self.graph = defaultdict(list)
        self.d = 0.85 #d是阻尼系数，一般设置为0.85
        self.min_diff = 1e-5 #设定收敛阈值

    #添加节点之间的边
    def addEdge(self, start, end, weight):
        self.graph[start].append((start, end, weight))
        self.graph[end].append((end, start, weight))

    #节点排序
    def rank(self):
        #默认初始化权重
        weight_deault = 1.0 / (len(self.graph) or 1.0)
        #nodeweight_dict, 存储节点的权重
        nodeweight_dict = defaultdict(float)
        #outsum，存储节点的出度权重
        outsum_node_dict = defaultdict(float)
        #根据图中的边，更新节点权重
        for node, out_edge in self.graph.items():
            #是 [('是', '全国', 1), ('是', '调查', 1), ('是', '失业率', 1), ('是', '城镇', 1)]
            nodeweight_dict[node] = weight_deault
            outsum_node_dict[node] = sum((edge[2] for edge in out_edge), 0.0)
        #初始状态下的textrank重要性权重
        sorted_keys = sorted(self.graph.keys())
        #设定迭代次数，
        step_dict = [0]
        for step in range(1, 1000):
            for node in sorted_keys:
                s = 0
                #计算公式：(edge_weight/outsum_node_dict[edge_node])*node_weight[edge_node]
                for e in self.graph[node]:
                    #e[2]=1
                    s += e[2] / outsum_node_dict[e[1]] * nodeweight_dict[e[1]]
                #计算公式：(1-d) + d*s
                nodeweight_dict[node] = (1 - self.d) + self.d * s
            step_dict.append(sum(nodeweight_dict.values()))

            if abs(step_dict[step] - step_dict[step - 1]) <= self.min_diff:
                break

        #利用Z-score进行权重归一化，也称为离差标准化，是对原始数据的线性变换，使结果值映射到[0 - 1]之间。
        #先设定最大值与最小值均为系统存储的最大值和最小值
        (min_rank, max_rank) = (sys.float_info[0], sys.float_info[3])
        for w in nodeweight_dict.values():
            if w < min_rank:
                min_rank = w
            if w > max_rank:
                max_rank = w

        for n, w in nodeweight_dict.items():
            nodeweight_dict[n] = (w - min_rank/10.0) / (max_rank - min_rank/10.0)

        return nodeweight_dict

class Docrank:
    def __init__(self):
        self.trainfile = r'D:\浏览器下载\ImportantEventExtractor-master!!!!!!!!!!!!!!!\news'
        self.storypath = r'D:\浏览器下载\ImportantEventExtractor-master!!!!!!!!!!!!!!!\story'
        self.doc_dict = self.seg_files()

    '''对训练文本进行分词等预处理，并为每篇文章维护一个词频字典'''
    def seg_files(self):
        doc_dict = {}
        for root, dirs, files in os.walk(self.trainfile):
            for file in files:
                filepath = os.path.join(root, file)
                #print(filepath)
                with open(filepath,"r",encoding='utf8')as f:
                    word_list = [w.word for w in pseg.cut(f.read()) if w.flag[0] not in ['x', 'w', 'u', 'c', 'p', 'q', 't', 'f', 'd'] and len(w.word) > 1]
                #print(word_list)
                word_dict = {i[0]: i[1] for i in Counter(word_list).most_common() if i[1] > 1}
                doc_dict[file] = word_dict
        #print(doc_dict["2015-10-12 12_54_49@砸一亿投向高端，中兴AXON天机是个什么鬼.txt"])
        return doc_dict

    '''为文章维护一个textrank表'''
    def doc_graph(self):
        g = textrank_graph()
        cm = defaultdict(int)
        for doc1, word_dict1 in self.doc_dict.items():
            for doc2, word_dict2 in self.doc_dict.items():
                if doc1 == doc2:
                    continue
                score = self.calculate_weight(word_dict1, word_dict2)
                pair = tuple((doc1, doc2))
                if score > 0:
                    cm[(pair)] = score
        #有了相似性就可以构造无向有权图进行TextRank排序
        for terms, w in cm.items():
            #添加这两个节点之间的边
            g.addEdge(terms[0], terms[1], w)
        nodes_rank = g.rank()
        #print(nodes_rank)#-->'2015-10-12 12_54_49@砸一亿投向高端，中兴AXON天机是个什么鬼.txt': 0.3078209862061127 
        nodes_rank = sorted(nodes_rank.items(), key=lambda asd:asd[1], reverse=True)

        return nodes_rank[:]

    '''计算文章之间的相关性'''
    def calculate_weight(self, word_dict1, word_dict2):
        score = 0
        #基于共词算法，计算新闻报道之间的相关性
        interwords = set(list(word_dict1.keys())).intersection(set(list(word_dict2.keys())))
        #print(interwords)
        for word in interwords:
            score += round(math.tanh(word_dict1.get(word)/word_dict2.get(word)))
        return score

    '''将同一时间的文章按照重要性进行排序'''
    def timeline(self, nodes_rank):
        f_story = open(r'D:\浏览器下载\ImportantEventExtractor-master!!!!!!!!!!!!!!!\timelines.txt', 'w+')
        f_important = open(r'D:\浏览器下载\ImportantEventExtractor-master!!!!!!!!!!!!!!!\important_doc.txt', 'w+')
        date_dict = {}
        timelines = {}
        for item in nodes_rank:
            f_important.write(item[0] + ' ' + str(item[1]) + '\n')
            doc = item[0]
            #观察文件名的前边就是日期
            date = int(item[0].split('@')[0].split(' ')[0].replace('-',''))
            if date not in date_dict:
                date_dict[date] = {}
                date_dict[date][doc] = item[1]
            else:
                date_dict[date][doc] = item[1]

        for date, doc_dict in date_dict.items():
            f = open(os.path.join(self.storypath, str(date)), 'w+')
            doc_dict = sorted(doc_dict.items(), key = lambda asd:asd[1], reverse=True)
            #重要性超过一定的阈值
            if doc_dict[0][1] > 0.4:
                timelines[date] = [str(doc_dict[0][0]), str(doc_dict[0][1])]
            for i in doc_dict:
                f.write(i[0] + "\t" + str(i[1]) + '\n')
            f.close()

        for i in sorted(timelines.items(), key=lambda asd:asd[0], reverse=False):
            f_story.write(str(i[0]) + ' ' + ' '.join(i[1]) + '\n')

        f_story.close()
        f_important.close()


        return timelines




handler = Docrank()
result = handler.doc_graph()
timelines = handler.timeline(result)





defaultdict(<class 'float'>, {'2015-10-12 12_54_49@砸一亿投向高端，中兴AXON天机是个什么鬼.txt': 0.3078209862061127, '2015-10-13 17_12_42@这才是国货：中兴自主处理器敲定！OS也来了.txt': 0.11333673765948264, '2015-12-16 14_06_08@中兴微电子：小心，我也有64位大小核芯片了.txt': 0.168904521995531, '2016-01-25 19_24_05@iPhone 6S天冷关机！中兴：零下30°无压力.txt': 0.06524200174612262, '2016-04-06 12_44_20@中兴换帅风波背后，美国：怪我咯.txt': 0.5503254129215214, '2017-08-21 13_20_29@中兴手机的困境，为啥国外混得比国内好？.txt': 0.2725728660027879, '2017-10-19 13_44_45@手握全球优质资源，中兴手机却打出这样的烂牌？.txt': 0.12123513049504214, '2017-12-16 11_47_46@中兴通讯员工坠亡始末解读：从富士康到中兴都在跳楼的原因.txt': 0.3794460284060379, '2017-12-18 09_19_59@对中兴工程师坠楼的思考：四十岁失业并不可怕.txt': 0.29250338667803133, '2017-12-26 12_36_05@中兴领主强势登陆尼日利亚全面挺进海外高端市场.txt': 0.04697677189769911, '2018-01-02 11_05_47@中兴通讯的2017，由这三大亮点组成？.txt': 0.335076572682672, '2018-01-19 10_22@中兴的2017：被罚数十亿，外挨打内求救，新品挑战多.txt': 0.3366672355897794, '2018-01-25 05_32_00@1用户对中兴AxonM整体配置评价很高.txt': 0.35032129288018243, '2018-01-25 13_13_43@痛定思痛之后，中兴仍难“中兴”.txt': 0.06626867356528447, '20

In [2]:
# 采用TextRank方法提取文本关键词
import sys
import pandas as pd
import jieba.analyse
# 处理标题和摘要，提取关键词
def getKeywords_textrank(data,topK):
    idList,titleList,abstractList = data['id'],data['title'],data['abstract']
    ids, titles, keys = [], [], []
    for index in range(len(idList)):
        text = '%s。%s' % (titleList[index], abstractList[index]) # 拼接标题和摘要
        jieba.analyse.set_stop_words("data/stopWord.txt") # 加载自定义停用词表
        print ("\"",titleList[index],"\"" , " 10 Keywords - TextRank :")
        keywords = jieba.analyse.textrank(text, topK=topK, allowPOS=('n','nz','v','vd','vn','l','a','d'))  # TextRank关键词提取，词性筛选
        word_split = " ".join(keywords)
        print(word_split)
        keys.append(word_split.encode("utf-8"))
        ids.append(idList[index])
        titles.append(titleList[index])

    result = pd.DataFrame({"id": ids, "title": titles, "key": keys}, columns=['id', 'title', 'key'])
    return result

    
def main():
    dataFile = 'data/sample_data.csv'
    data = pd.read_csv(dataFile)
    result = getKeywords_textrank(data,10)
    result.to_csv("result/keys_TextRank.csv",index=False)


In [18]:
import jieba.analyse
import pandas as pd
import numpy as np
#data=pd.read_csv("")
with open(r"D:\浏览器下载\learning-nlp-master\chapter-5\stopword.txt","r",encoding='utf8')as f:
    stopwords=f.readlines()
    #print(stopwords)
def seg_content(text):
    contents=jieba.lcut(text)
    contents=[word for word in contents if word not in stopwords]
    print(contents)
    clean_contents="".join(contents)
    return clean_contents
text="百度提供了站内搜索，北京创新技术有限公司"   
clean_content=seg_content(text)
jieba.analyse.textrank(clean_content,topK=3)
#data["content"]=data["content"].map(lambda x:seg_content(x))

['百度', '提供', '了', '站内搜索', '，', '北京', '创新', '技术', '有限公司']


['北京', '有限公司', '技术']

In [ ]:
'''阿博茨科技项目之新闻主题句提取：改进textrank加入位置权重
1.分句
2.分词
3.tfidf，平均值计算句子的向量表征（当然也可以用Word2vec,glove等，不过这里公司提供了内部tfidf，哈哈省事）
4.构造句子相似度矩阵：sim_mat=np.zeros((len(sentence),len(sentence)))，
改进点：句子位置、（余弦）句子相似度、关键词覆盖率（关键词已给出）进行加权作为分数。
实验结果权重λ1 = 0.13，λ2 = 0.19，λ3 = 0.38。
5.在进行下一步之前，我们先将相似性矩阵sim_mat转换为图结构。这个图的节点为句子，
边用句子之间的相似性分数表示。在这个图上，我们将应用PageRank算法来得到句子排名。
[也可以自己写个类函数textrank_graph]
import network as nx
nx_graph=nx.from_numpy_array(sim_mat)
scores=nx.pagerank(nx_graph)
6.PageRank，textrank思想，输出topk，平均每个含3个主题句,【k大点可以生成摘要了】，k=3作为主题句候选
ranked_sentences=sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
for i in range(k):
print(ranked_sentences[i][1])
评估：
1.单纯用tfidf计算句子相似度排序，二者挑选的句子与人工标注的主题句进行比较，以及准确率、召回率和F-score评估
P = Scorrect/Sextract × 100% ( 11) ,
Ｒ = Scorrect/Sstandar × 100% ( 12) 
F = 2PＲ/(P + Ｒ) × 100% ( 13)
其中，Scorrect 是准确抽取的关键事件主题句数目; Sextract是所有抽取的关键事件主题句数目; Sstandar是所
有人工标注的关键事件主题句数目。
2.textrank不考虑其他，jieba.analyse.textrank输出topk结果。

'''
'''这里定义一个类函数实现textrank，但简单的实现textrank，没有改进'''
#!/usr/bin/env python3
# coding: utf-8
# File: doc_rank.py
# Author: lhy<lhy_in_blcu@126.com,https://huangyong.github.io>
# Date: 18-7-9

import jieba.posseg as pseg
import os
from collections import Counter
from collections import defaultdict
import sys
import math


class textrank_graph:
    def __init__(self):
        self.graph = defaultdict(list)
        self.d = 0.85 #d是阻尼系数，一般设置为0.85
        self.min_diff = 1e-5 #设定收敛阈值

    #添加节点之间的边
    def addEdge(self, start, end, weight):
        self.graph[start].append((start, end, weight))
        self.graph[end].append((end, start, weight))

    #节点排序
    def rank(self):
        '''默认初始化权重，加权textrank改的就是这里，w=λ1*句子相似度a+λ2*关键词覆盖度b+λ3*位置值c,
        #比如句子是在句首，c=1,a=45,b=2,那么两个句子的边权重加权求和'''
        weight_deault = 1.0 / (len(self.graph) or 1.0)
        #nodeweight_dict, 存储节点的权重
        nodeweight_dict = defaultdict(float)
        #outsum，存储节点的出度权重
        outsum_node_dict = defaultdict(float)
        #根据图中的边，更新节点权重
        for node, out_edge in self.graph.items():
            #是 [('是', '全国', 1), ('是', '调查', 1), ('是', '失业率', 1), ('是', '城镇', 1)]
            nodeweight_dict[node] = weight_deault
            outsum_node_dict[node] = sum((edge[2] for edge in out_edge), 0.0)
        #初始状态下的textrank重要性权重
        sorted_keys = sorted(self.graph.keys())
        #设定迭代次数，
        step_dict = [0]
        for step in range(1, 1000):
            for node in sorted_keys:
                s = 0
                #计算公式：(edge_weight/outsum_node_dict[edge_node])*node_weight[edge_node]
                for e in self.graph[node]:
                    s += e[2] / outsum_node_dict[e[1]] * nodeweight_dict[e[1]]
                #计算公式：(1-d) + d*s
                nodeweight_dict[node] = (1 - self.d) + self.d * s
            step_dict.append(sum(nodeweight_dict.values()))

            if abs(step_dict[step] - step_dict[step - 1]) <= self.min_diff:
                break

        #利用Z-score进行权重归一化，也称为离差标准化，是对原始数据的线性变换，使结果值映射到[0 - 1]之间。
        #先设定最大值与最小值均为系统存储的最大值和最小值
        (min_rank, max_rank) = (sys.float_info[0], sys.float_info[3])
        for w in nodeweight_dict.values():
            if w < min_rank:
                min_rank = w
            if w > max_rank:
                max_rank = w

        for n, w in nodeweight_dict.items():
            nodeweight_dict[n] = (w - min_rank/10.0) / (max_rank - min_rank/10.0)

        return nodeweight_dict

class Docrank:
    def __init__(self):
        self.trainfile = r'D:\浏览器下载\ImportantEventExtractor-master!!!!!!!!!!!!!!!\news'
        self.storypath = r'D:\浏览器下载\ImportantEventExtractor-master!!!!!!!!!!!!!!!\story'
        self.doc_dict = self.seg_files()

    '''对训练文本进行分词等预处理，并为每篇文章维护一个词频字典'''
    def seg_files(self):
        doc_dict = {}
        for root, dirs, files in os.walk(self.trainfile):
            for file in files:
                filepath = os.path.join(root, file)
                #print(filepath)
                with open(filepath,"r",encoding='utf8')as f:
                    word_list = [w.word for w in pseg.cut(f.read()) if w.flag[0] not in ['x', 'w', 'u', 'c', 'p', 'q', 't', 'f', 'd'] and len(w.word) > 1]
                #print(word_list)
                word_dict = {i[0]: i[1] for i in Counter(word_list).most_common() if i[1] > 1}
                doc_dict[file] = word_dict
        #print(doc_dict["2015-10-12 12_54_49@砸一亿投向高端，中兴AXON天机是个什么鬼.txt"])
        return doc_dict

    '''为文章维护一个textrank表'''
    def doc_graph(self):
        g = textrank_graph()
        cm = defaultdict(int)
        for doc1, word_dict1 in self.doc_dict.items():
            for doc2, word_dict2 in self.doc_dict.items():
                if doc1 == doc2:
                    continue
                score = self.calculate_weight(word_dict1, word_dict2)
                pair = tuple((doc1, doc2))
                if score > 0:
                    cm[(pair)] = score
        #有了相似性就可以构造无向有权图进行TextRank排序
        for terms, w in cm.items():
            #添加这两个节点之间的边
            g.addEdge(terms[0], terms[1], w)
        nodes_rank = g.rank()
        #print(nodes_rank)#-->'2015-10-12 12_54_49@砸一亿投向高端，中兴AXON天机是个什么鬼.txt': 0.3078209862061127 
        nodes_rank = sorted(nodes_rank.items(), key=lambda asd:asd[1], reverse=True)

        return nodes_rank[:]

    '''计算文章之间的相关性'''
    def calculate_weight(self, word_dict1, word_dict2):
        score = 0
        #基于共词算法，计算新闻报道之间的相关性
        interwords = set(list(word_dict1.keys())).intersection(set(list(word_dict2.keys())))
        #print(interwords)
        for word in interwords:
            score += round(math.tanh(word_dict1.get(word)/word_dict2.get(word)))
        return score

    '''将同一时间的文章按照重要性进行排序'''
    def timeline(self, nodes_rank):
        f_story = open(r'D:\浏览器下载\ImportantEventExtractor-master!!!!!!!!!!!!!!!\timelines.txt', 'w+')
        f_important = open(r'D:\浏览器下载\ImportantEventExtractor-master!!!!!!!!!!!!!!!\important_doc.txt', 'w+')
        date_dict = {}
        timelines = {}
        for item in nodes_rank:
            f_important.write(item[0] + ' ' + str(item[1]) + '\n')
            doc = item[0]
            #观察文件名的前边就是日期
            date = int(item[0].split('@')[0].split(' ')[0].replace('-',''))
            if date not in date_dict:
                date_dict[date] = {}
                date_dict[date][doc] = item[1]
            else:
                date_dict[date][doc] = item[1]

        for date, doc_dict in date_dict.items():
            f = open(os.path.join(self.storypath, str(date)), 'w+')
            doc_dict = sorted(doc_dict.items(), key = lambda asd:asd[1], reverse=True)
            #重要性超过一定的阈值
            if doc_dict[0][1] > 0.4:
                timelines[date] = [str(doc_dict[0][0]), str(doc_dict[0][1])]
            for i in doc_dict:
                f.write(i[0] + "\t" + str(i[1]) + '\n')
            f.close()

        for i in sorted(timelines.items(), key=lambda asd:asd[0], reverse=False):
            f_story.write(str(i[0]) + ' ' + ' '.join(i[1]) + '\n')

        f_story.close()
        f_important.close()


        return timelines




handler = Docrank()
result = handler.doc_graph()
timelines = handler.timeline(result)




